<a href="https://colab.research.google.com/github/Yiifeiii/IT1244/blob/main/Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
train_df = pd.read_csv('/content/Train_Set.csv')
train_df_ = pd.read_csv('/content/Train_Set.csv')

# TEST DATA
test_df = pd.read_csv('/content/Test_Set.csv')
test_df_ = pd.read_csv('/content/Test_Set.csv')

# Display the DataFrame
train_df.head()
train_df.describe()
train_df['class_label'].value_counts()


FileNotFoundError: [Errno 2] No such file or directory: '/content/Train_Set.csv'

In [ ]:
# Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# General information about the dataset
train_info = train_df.describe()
test_info = test_df.describe()

# Plot distribution of classes
plt.figure(figsize=(10, 6))
sns.countplot(x='class_label', data=train_df)
plt.title('Distribution of Classes in Training Set')
plt.xticks(rotation=45)
plt.show()

# Plot distribution of a few feature columns to understand their spread
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
for i, col in enumerate(['length_51', 'length_200', 'length_400']):
    sns.histplot(train_df[col], ax=axes[i], kde=True)
    axes[i].set_title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

train_info, test_info


1. The class distribution in the training set shows that the dataset is class imbalanced, as seen in the distribution of classes.

2. The features, represented by the lengths of DNA fragments (e.g., length_51, length_200, length_400), show varied distributions. Some have a right-skewed distribution, indicating a higher frequency of lower values.

3. The general statistics for both training and test datasets indicate ranges of normalized frequencies for different lengths, with the means and standard deviations varying across different lengths. This variation suggests diversity in the DNA fragment lengths' normalized frequencies, which could be crucial for classifying the sample classes.

label healthy as 0 and cancer as 1 and Standardisation

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Recategorize the class_label in both datasets
train_df['class_label'] = train_df['class_label'].apply(lambda x: 0 if x == 'healthy' else 1)
test_df['class_label'] = test_df['class_label'].apply(lambda x: 0 if x == 'healthy' else 1)

# Separate features and labels
X_train = train_df.drop('class_label', axis=1)
y_train = train_df['class_label']
X_test = test_df.drop('class_label', axis=1)
y_test = test_df['class_label']

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Perform PCA on the standardized train_set
pca = PCA(n_components=10)  # Keep the first 10 principal components
X_train_pca = pca.fit_transform(X_train_scaled)

X_train_scaled.shape, X_train_pca.shape, y_train.value_counts(), y_test.value_counts()


split the healthy and early stage cancer

In [ ]:
# Creating a new dataset with only "healthy" and "early stage" cancer samples
train_df_subset = train_df_[(train_df_['class_label'] == 'healthy') | (train_df_['class_label'] == 'early stage cancer')]
test_df_subset = test_df_[(test_df_['class_label'] == 'healthy') | (test_df_['class_label'] == 'early stage cancer')]

# Recategorize the class_label in both datasets
train_df_subset['class_label'] = train_df_subset['class_label'].apply(lambda x: 0 if x == 'healthy' else 1)
test_df_subset['class_label'] = test_df_subset['class_label'].apply(lambda x: 0 if x == 'healthy' else 1)

# Separate features and labels
X_train_subset = train_df_subset.drop('class_label', axis=1)
y_train_subset = train_df_subset['class_label']
X_test_subset = test_df_subset.drop('class_label', axis=1)
y_test_subset = test_df_subset['class_label']

# Standardize the features
scaler = StandardScaler()
X_train_subset_scaled = scaler.fit_transform(X_train_subset)
X_test_subset_scaled = scaler.transform(X_test_subset)

y_train_subset.value_counts(), y_test_subset.value_counts()


PCA for feature selection for cancer VS healthy

In [ ]:
from sklearn.decomposition import PCA

# Initialize PCA
pca = PCA(n_components=10)  # You can choose the number of components as needed

# Perform PCA on training data
X_train_pca = pca.fit_transform(X_train_scaled)
X_train_pca_df = pd.DataFrame(data=X_train_pca, columns=[f'PC{i+1}' for i in range(10)])
# Transform test data using the same PCA transformation
X_test_pca = pca.transform(X_test_scaled)
X_test_pca_df = pd.DataFrame(data=X_test_pca, columns=[f'PC{i+1}' for i in range(10)])

X_train_pca_df.head(), pca.explained_variance_ratio_

PCA for feature selection for healthy VS early stage cancer  

In [ ]:
# Perform PCA on early stage trainig data
X_train_pca_early = pca.fit_transform(X_train_subset_scaled)
X_train_pca_early_df = pd.DataFrame(data=X_train_pca_early, columns=[f'PC{i+1}' for i in range(10)])
# Transform early stage test data using the same PCA transformation
X_test_pca_early = pca.fit_transform(X_test_subset_scaled)
X_test_pca_early_df = pd.DataFrame(data=X_test_pca_early, columns=[f'PC{i+1}' for i in range(10)])

X_train_pca_early_df.head(), pca.explained_variance_ratio_

Train healthy VS cancer with multilayer neural network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Initialize and train the basic multilayer neural network model
#mlp = MLPClassifier(hidden_layer_sizes=(1,), max_iter=1000, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(1000,), max_iter=1000, random_state=42)
mlp.fit(X_train_pca, y_train)

# Predict on the transformed test set (after applying the same standardization and PCA transformation)
y_pred = mlp.predict(X_test_pca)
y_pred_proba = mlp.predict_proba(X_test_pca)[:, 1]  # Probability estimates for AUROC

# Calculate precision, recall, F1 score, and AUROC
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auroc = roc_auc_score(y_test, y_pred_proba)

precision, recall, f1, auroc

precision: 0.96
recall: 0.99
f1: 0.97
auroc: 0.81

Train healthy VS early stage cancer with multilayer neural network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Initialize and train the basic multilayer neural network model
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp.fit(X_train_pca_early, y_train_subset)

# Predict on the transformed test set (after applying the same standardization and PCA transformation)
y_pred_early = mlp.predict(X_test_pca_early)
y_pred_proba_early = mlp.predict_proba(X_test_pca_early)[:, 1]  # Probability estimates for AUROC

# Calculate precision, recall, F1 score, and AUROC
precision = precision_score(y_test_subset, y_pred_early)
recall = recall_score(y_test_subset, y_pred_early)
f1 = f1_score(y_test_subset, y_pred_early)
auroc = roc_auc_score(y_test_subset, y_pred_proba_early)

precision, recall, f1, auroc

precision: 0.90
recall: 0.70
f1: 0.79
auroc: 0.56

add error function to tackle imbalanced data

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, f1_score

# Define the MLP model with hyperbolic tangent activation function and a learning rate of 0.1

# Define a range of hidden layer sizes to test (for simplicity, we vary the number of layers,
# each with a fixed size of 100 neurons)
hidden_layer_sizes = [(100,), (100, 100), (100, 100, 100)]  # Example: 1 layer, 2 layers, 3 layers

# Setup grid search parameters
param_grid = {
    'hidden_layer_sizes': hidden_layer_sizes,
}

# Define MLPClassifier with fixed parameters
mlp2 = MLPClassifier(activation='tanh', solver='adam', learning_rate_init=0.1, max_iter=1000, random_state=42)

# Define the cross-validation scheme and grid search
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=mlp2, param_grid=param_grid, cv=cv, scoring=make_scorer(f1_score), n_jobs=-1)

# Fit grid search (this can be time-consuming)
grid_search.fit(X_train_pca, y_train)

# Best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score


F1 score: 0.99


In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, f1_score

# Define the MLP model with hyperbolic tangent activation function and a learning rate of 0.1

# Define a range of hidden layer sizes to test (for simplicity, we vary the number of layers,
# each with a fixed size of 100 neurons)
hidden_layer_sizes = [(100,), (100, 100), (100, 100, 100)]  # Example: 1 layer, 2 layers, 3 layers

# Setup grid search parameters
param_grid = {
    'hidden_layer_sizes': hidden_layer_sizes,
}

# Define MLPClassifier with fixed parameters
mlp2 = MLPClassifier(activation='tanh', solver='adam', learning_rate_init=0.1, max_iter=1000, random_state=42)

# Define the cross-validation scheme and grid search
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=mlp2, param_grid=param_grid, cv=cv, scoring=make_scorer(f1_score), n_jobs=-1)

# Fit grid search (this can be time-consuming)
grid_search.fit(X_train_pca_early, y_train_subset)

# Best parameters and score
best_params_early = grid_search.best_params_
best_score_early = grid_search.best_score_

best_params_early, best_score_early


F1 score: 0.97